<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/802%20code/bert-base-uncased-paradetox-1Token-Split-MASK-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers[torch]

In [10]:
!pip install huggingface_hub

In [11]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from datasets import load_dataset
from transformers import BertForMaskedLM
from transformers import BertTokenizer

dataset = load_dataset("HamdanXI/paradetox-1Token-Split-MASK")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3784 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/811 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/811 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
train_max_token_length_input = max(len(tokenizer.encode(item)) for item in dataset['train']['labels'])
train_max_token_length_label = max(len(tokenizer.encode(item)) for item in dataset['train']['input'])

valid_max_token_length_input = max(len(tokenizer.encode(item)) for item in dataset['validation']['labels'])
valid_max_token_length_label = max(len(tokenizer.encode(item)) for item in dataset['validation']['input'])

test_max_token_length_input = max(len(tokenizer.encode(item)) for item in dataset['test']['labels'])
test_max_token_length_label = max(len(tokenizer.encode(item)) for item in dataset['test']['input'])

if train_max_token_length_input > train_max_token_length_label:
    highest_length_train = train_max_token_length_input
else:
    highest_length_train = train_max_token_length_label

if valid_max_token_length_input > valid_max_token_length_label:
    highest_length_valid = valid_max_token_length_input
else:
    highest_length_valid = valid_max_token_length_label

if test_max_token_length_input > test_max_token_length_label:
    highest_length_test = test_max_token_length_input
else:
    highest_length_test = test_max_token_length_label

if highest_length_train > highest_length_valid:
    highest_length = highest_length_train
else:
    highest_length = highest_length_valid

if highest_length_test > highest_length:
    highest_length = highest_length_test

print(highest_length)

30


In [6]:
def prepare_labels(examples):
    tokenized_inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=highest_length)
    tokenized_labels = tokenizer(examples["labels"], padding="max_length", truncation=True, max_length=highest_length)

    labels = []
    for i, label in enumerate(tokenized_labels["input_ids"]):
        label_with_mask = [-100] * len(label)  # Initialize labels with -100
        input_ids = tokenized_inputs["input_ids"][i]
        mask_indices = [j for j, token in enumerate(input_ids) if token == tokenizer.mask_token_id]

        for mask_index in mask_indices:
            if mask_index < len(label):
                label_with_mask[mask_index] = label[mask_index]

        labels.append(label_with_mask)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [23]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3784
    })
    validation: Dataset({
        features: ['labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 811
    })
    test: Dataset({
        features: ['labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 811
    })
})

In [24]:
tokenized_datasets['train']['input'][0]

'we have a good health care education system, but our health care delivery system [MASK]'

In [7]:
tokenized_datasets = dataset.map(prepare_labels, batched=True)

Map:   0%|          | 0/3784 [00:00<?, ? examples/s]

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="bert-base-uncased-paradetox-1Token-Split-MASK-v2",
    num_train_epochs=30,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

In [16]:
trainer.train()

Step,Training Loss
100,0.608400
200,0.473500
300,0.445100
400,0.371100
500,0.470500
600,0.370000
700,0.516100
800,0.663200
900,0.670100
1000,0.595700


TrainOutput(global_step=14190, training_loss=0.3323149203918113, metrics={'train_runtime': 1133.6185, 'train_samples_per_second': 100.14, 'train_steps_per_second': 12.517, 'total_flos': 1750723262640000.0, 'train_loss': 0.3323149203918113, 'epoch': 30.0})

In [18]:
tokenizer.push_to_hub("bert-base-uncased-paradetox-1Token-Split-MASK-v2")

CommitInfo(commit_url='https://huggingface.co/HamdanXI/bert-base-uncased-paradetox-1Token-Split-MASK-v2/commit/3b6ec1ab574a49fda662a6979a01cee79679437c', commit_message='Upload tokenizer', commit_description='', oid='3b6ec1ab574a49fda662a6979a01cee79679437c', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
trainer.push_to_hub("bert-base-uncased-paradetox-1Token-Split-MASK-v2")

'https://huggingface.co/HamdanXI/bert-base-uncased-paradetox-1Token-Split-MASK-v2/tree/main/'